In [ ]:
import pandas as pd
import numpy as np
import re
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
def read_excel_sheets(xls_path):
    """Read all sheets of an Excel workbook and return a single DataFrame"""
    print(f'Loading {xls_path} into pandas')
    xl = pd.ExcelFile(xls_path)
    clslist = []
    sheetnames =[]
    columns = None
    for idx, name in enumerate(xl.sheet_names):
        print(f'Reading sheet #{idx}: {name}')
        sheet = xl.parse(name)
        # Assume index of existing data frame when appended
        clslist.append(sheet)
        sheetnames.append(name)
    return clslist, sheetnames
cls2020, names = read_excel_sheets("C:/Users/parke/OneDrive/Documents/GitHub/Attollo/MasterSheet CSVS/Scholar 2018-2019 Master Contact Sheet.xlsx")
for i in range(len(names)):
    names[i] = names[i].rstrip()

## See the differences in columns for each school

In [ ]:
def pcol(df):
    print(df.columns.size, df.columns)
for i in cls2020:
    pcol(i)

## Clean columns

In [ ]:
def cleancolumns(df):
    #removes unnamed
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.columns = df.columns.str.replace('\(P', '(G', regex=True)
    if 'Staff Initials' in df.columns:
        del df['Staff Initials']
    if 'Staff initials' in df.columns:
        del df['Staff initials']
    if 'Staff Initial' in df.columns:
        del df['Staff Initial']
    if 'Last Update' in df.columns:
        del df['Last Update']
    if 'Last Updated' in df.columns:
        del df['Last Updated']
    return df
for i in range(9):
    cls2020[i]=cleancolumns(cls2020[i])

## Modify honorary

In [ ]:
cls2020[7].head()
cls2020[7]['Guardian 1 First Name'] = cls2020[7]['Guardian 1 Name']
del cls2020[7]['Guardian 1 Name']
cls2020[7] = cls2020[7].rename(columns={'HS Graduation Year':'gradyear'})
cls2020[7]

## Make all first 9 into same df

In [ ]:
allcols = cls2020[3].columns

#add columns that dont exist
count =1
for df in cls2020[:9]: 
    print(names[count-1], '-----')
    for c in allcols:
        if c not in df.columns:
            df[c]=np.nan
    df=df.reindex(allcols, axis ="columns")
    count+=1
allcols

#concatenate into one dataframe
schools = names[:9]
alldf= pd.DataFrame()
for i in range(9):
    cls2020[i]["School"] = schools[i]
alldf = pd.concat(cls2020[:9], ignore_index =True)

## Rename all columns

In [ ]:
orderedcols = {'Student First Name':'fname', 'Student Last Name':'lname', 'Student Email':'email', 
               'Student Phone':'phonenum','School':'schoolid','Shirt Size':'shirt','Short Size':'short',
               'Candy':'favcandy','Guardian 1 First Name':'guard1fname','Guardian 1 Last Name':'guard1lname',
               'Guardian 1 Phone':'guard1phonenum','Guardian 1 Email':'guard1email','Guardian 1 Occupation':'guard1occ',
               'Shirt Size (G1)':'guard1shirt','Guardian 2 First Name':'guard2fname','Guardian 2 Last Name':'guard2lname',
               'Guardian 2 Phone':'guard2phonenum','Guardian 2 Email':'guard2email','Guardian 2 Occupation':'guard2occ',
               'Shirt Size (G2)':'guard2shirt'}
def renamealldf(df):
    df.rename(orderedcols, axis=1, inplace=True)
    return df
renamedalldf= renamealldf(alldf)
renamedalldf.head()

## Add in Data from Medical Sheet
- could honestly get more but not sure how to match since not the same last name

In [ ]:
cls2020[9] = cls2020[9].rename(columns ={'School District': 'schoolid', 'Last Name':'lname'})
cls2020[9] = cleancolumns(cls2020[9])
cls2020[9] = cls2020[9].astype(str)
cls2020[9] = cls2020[9].applymap(lambda x: re.sub(r'^-$', str(np.NaN), x))
cls2020[9]['comments'] =str(np.nan)
algcheck = cls2020[9][cls2020[9]["Allergies"] != str(np.NaN)].index.tolist()
for i in algcheck:
    cls2020[9].iloc[i, :]["comments"] = "Allergies: " + cls2020[9].iloc[i, :]['Allergies']

medcheck = cls2020[9][cls2020[9]["Medical Conditions"] != str(np.NaN)].index.tolist()
for i in medcheck:
    x = cls2020[9].iloc[i, :]
    if x["comments"] != str(np.NaN):
        x["comments"] = x["comments"] + '; Medical Conditions: ' + x["Medical Conditions"]
    else:
        x["comments"] = 'Medical Conditions: ' + x['Medical Conditions'] 
    cls2020[9].iloc[i, :] = x
    
drugcheck = cls2020[9][cls2020[9]["Medicines"] != str(np.NaN)].index.tolist()
for i in drugcheck:
    x = cls2020[9].iloc[i, :]
    if x["comments"] != str(np.NaN):
        x["comments"] = x["comments"] + '; Medicines: ' + x["Medicines"]
    else:
        x["comments"] = 'Medicines: ' + x['Medicines'] 
    cls2020[9].iloc[i, :] = x
    
notecheck = cls2020[9][cls2020[9]["Notes"] != str(np.NaN)].index.tolist()
for i in notecheck:
    x = cls2020[9].iloc[i, :]
    if x["comments"] != str(np.NaN):
        x["comments"] = x["comments"]+ '; Notes: ' + x['Notes'] 
    else:
        x["comments"] = 'Notes: ' + x["Notes"]
    cls2020[9].iloc[i, :] = x
del cls2020[9]["Allergies"] 
del cls2020[9]["Medical Conditions"]   
del cls2020[9]["Medicines"] 
del cls2020[9]["Notes"]   
cls2020[9]

# comment = "Allergies: " + cls2020[9]['Allergies'] +', Medical Conditions: ' + \
#                          cls2020[9]['Medical Conditions'] + cls2020[9]['Notes'] + ', Medicines: ' + cls2020[9]['Medicines']
# cls2020[9]['comments'] = comment

In [ ]:
combined = pd.merge(renamedalldf, cls2020[9], on=['lname','schoolid'], how='left')
del combined['First Name']
combined.columns

## Clean all Data

In [ ]:
def transform(dforig):
    df = dforig.copy()
    
    df = df.astype(str)
    df = df.applymap(lambda x: re.sub(r'^-$', str(np.NaN), x))
    #make all lower email
    df['email'] = df['email'].str.lower()
    df['guard1email'] = df['guard1email'].str.lower()
    df['guard2email'] = df['guard2email'].str.lower()
    #remove all nonnumbers in phonenumbers
    df['phonenum'] = df['phonenum'].str.replace('-', '', regex=True).str.replace(' ', '', regex=True).str.replace('FBMessenger','', regex=True).str.replace('kevinsingh422@yahoo.com','', regex=True)
    df['guard1phonenum'] = df['guard1phonenum'].str.replace('-', '').str.replace(' ', '')
    df['guard2phonenum'] = df['guard2phonenum'].str.replace('-', '').str.replace(' ', '')
    # convert all phonenumbers to string
    df['phonenum'] = df['phonenum'].astype(str)    
    df['guard1phonenum'] = df['guard1phonenum'].astype(str)
    df['guard2phonenum'] = df['guard2phonenum'].astype(str)
    # remove all questions in sizes
    df['shirt'] = df['shirt'].str.replace('2X', 'XXL', regex=True).str.replace('3X', 'XXL', regex=True)
    df['short'] = df['short'].str.replace('2X', 'XXL', regex=True).str.replace('3X', 'XXL', regex=True).str.replace('4X', 'XXL', regex=True)
    df['guard1shirt'] = df['guard1shirt'].str.replace('2XL', 'XXL', regex=True).str.replace('2X', 'XXL', regex=True).str.replace('3X', 'XXL', regex=True).str.replace('4X','XXL')
    df['guard2shirt'] = df['guard2shirt'].str.replace('2XL', 'XXL', regex=True).str.replace('2X', 'XXL', regex=True).str.replace('3XL', 'XXL', regex=True).str.replace('3X','XXL').str.replace('4X','XXL').str.replace('5X','XXL')

    
    gradcheck = df[df["gradyear"] == str(np.nan)].index.tolist()
#     print(gradcheck)
    for i in gradcheck:
        df.iloc[i,:]['gradyear'] = 2020
    df['gradyear'] = df['gradyear'].astype(float)
    df['gradyear'] = df['gradyear'].astype(int)
    
    return df
cleancombined = transform(combined)

def intphone(df, var):
    for i in df.index:
        if df.loc[i,var] == str(np.nan):
            df.loc[i,var]== np.nan
        else:
            df.loc[i,var] = '+1'+df.loc[i,var]
intphone(cleancombined, 'phonenum')
intphone(cleancombined, 'guard1phonenum')
intphone(cleancombined, 'guard2phonenum')
#get rid of nan name
cleancombined = cleancombined[:-1]

## Write to csv

In [ ]:
#needed blank columns like transdf2021 to use same upload code
cleancombined['dob']= np.nan
cleancombined['student_ig']= np.nan
cleancombined['address']= np.nan
cleancombined['gender']= np.nan
cleancombined['race']= np.nan
cleancombined['guard1occ']= np.nan
cleancombined['guard2occ']= np.nan
cleancombined['emergcontact']= np.nan

filepath = 'attollo/basic/management/commands/cls2020.csv'
cleancombined.to_csv(filepath)

In [ ]:
filepath = 'attollo/basic/management/commands/cls2020.csv'
out = pd.read_csv(filepath)
out